In [38]:
#Подключение библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
import altair as alt

#Получение данных с сайта
url = "https://en.wikipedia.org/wiki/Solar_power_by_country"
id_table = 1
df_power = pd.read_html(url, header = 0, index_col = 0)[id_table]
df_power

#1 Сортировка данных по убыванию 3 столбца (% gen.)
df_power.sort_values('% gen.', ascending=False)

#2 Добавление к таблице столбца с новыми данными, полученным путем преобразований имеющихся
df_power['from the total Gen %'] = df_power.iloc[: , 0] / 1310.02 * 100
df_power

df_power['Gen (TWh) > 1'] = df_power['Gen (TWh)'] >= 1
df_power

#3 Выбор фрагмента таблицы по условию (по процентному росту солнечной мощности в годовом исчислении (% cap. growth))
print(df_power[df_power['% cap. growth'] > 40.])

#4 Выбор фрагмента таблицы по номерам строк и столбцов. Через iloc по номерам строк и столбцов, номера идут с нуля
print('данные из второй строки и второго столбца')
print(df_power.iloc[1 , 1])

print('данные из всех строк кроме пяти последних и из столбцов со второго по третий включительно')
print(df_power.iloc[: -5, 1 : 4])

print('данные из нечетных столбцов и третьей строки')
print(df_power.iloc[ 3, : : 2])

#5 Расчеты статистических показателей по сгруппированным данным

#Суммарная мощность в гигаваттах на конец года для стран, вырабатывающие больше 1 TWh и меньше 1 TWh
sum_stat = df_power.groupby(['Gen (TWh) > 1'])['% cap. growth'].sum()
print(sum_stat)
sum_stat.plot(kind = 'bar')

#Максимальный процент выработки солнечной энергии в странах, вырабатывающие больше 1 TWh и меньше 1 TWh
max_stat = df_power.groupby(['Gen (TWh) > 1'])['% gen.'].max()
print(max_stat)
max_stat.plot(kind = 'bar')

#Число выработки солнечной энергии ниже и выше 50 TWh и разделением по коэф. мощности
tot_stat = (df_power.groupby(['Cap. fac.', df_power['Gen (TWh)'] > 50])['Gen (TWh)'].count())
print(tot_stat)
tot_stat.plot(kind = 'bar')

#6  Построение гистограммы по данным
axes = df_power.iloc[ 1 : 25 , 0].plot(kind = 'bar', figsize=(24, 5), title='Гистограмма', color = 'cyan' )
for p in axes.patches:
    axes.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))
axes.grid(True)

#7 Построение круговой диаграммы по данным.
ax = df_power.iloc[1 : 11 , 0].plot(kind = 'pie', title='Топ-10 стран по выработке солнечной энергии', autopct="%1.1f%%", startangle=40)

#8 Построение интерактивного графика по данным.
output_notebook()
source = ColumnDataSource(data=df_power)
p = figure(x_axis_label="Gen (TWh)", y_axis_label="Cap (GW)")
p.line(x="Gen (TWh)", y="Cap. (GW)", source=source, line_width=2, color='blue')
p.circle(x="Gen (TWh)", y="Cap. (GW)", source=source, alpha=0.5, size = 8, color='cyan')
show(p)